In [55]:
import re, sys, copy, numpy as np, threading
from math import sqrt
from aoclib import list2string
from collections import defaultdict
from itertools import combinations
from operator import add, mul

lines = open('23.txt').read().strip()

program = [int(x) for x in lines.split(",")]

In [56]:
def icc(program, input_queue = None, pointer = 0): # intcode computer    
    memory = program.copy()
    for j in range(10000 - len(memory)):
        memory.append(0)
                      
    #outputs[network_address] = []
    alt_par_modes = False # parameter modes other than position mode are enabled / disabled by this boolean
    relative_base = 0

    i = pointer
    N_empty = 0
    while True:
        #print("Running", network_address)
        if alt_par_modes:
            alt_par_modes = False
            instruction_header = str(memory[i]).rjust(5, "0")
            opcode = int(instruction_header[-2:])
            par_modes = [int(x) for x in instruction_header[-3::-1]]
        else:
            opcode = memory[i]
            par_modes = [0, 0, 0]

        if opcode == 99:
            break
            
        idx1 = i + 1 if par_modes[0] == 1 else memory[i + 1]
        idx2 = i + 2 if par_modes[1] == 1 else memory[i + 2]
        idx3 = i + 3 if par_modes[2] == 1 else memory[i + 3]
        if par_modes[0] == 2:
            idx1 += relative_base
        if par_modes[1] == 2:
            idx2 += relative_base
        if par_modes[2] == 2:
            idx3 += relative_base
        
        match opcode:
            case 1 | 2:
                memory[idx3] = memory[idx1] + memory[idx2] if opcode == 1 else memory[idx1] * memory[idx2]
                i += 4
            case 3:
                if input_queue:
                    input_ = int(input_queue.pop(0))
                    #print(input_)
                else:
                    input_ = -1
                    #input_ = int(input_queue('Please provide some input: '))
                memory[idx1] = input_
                i += 2
                return None, i, memory
            case 4:
                #outputs[network_address].append(memory[idx1])
                i += 2
                return memory[idx1], i, memory
            case 5:
                i = memory[idx2] if memory[idx1] != 0 else i + 3
            case 6:
                i = memory[idx2] if memory[idx1] == 0 else i + 3
            case 7:
                memory[idx3] = 1 if memory[idx1] < memory[idx2] else 0
                i += 4
            case 8:
                memory[idx3] = 1 if memory[idx1] == memory[idx2] else 0
                i += 4
            case 9:
                relative_base += memory[idx1]
                i += 2
            case _:
                alt_par_modes = True
                continue
    return None, i, memory


In [57]:
# Part 1
def boot(outputs, pointers, memories, n = 50):
    for i in range(n):
        #print("booting", i)
        outputs[i], pointers[i], memories[i] = icc(program, [i])#input_queues[i], i)
        #threads.append(threading.Thread(target=task).start())
    return outputs, pointers, memories

# Init network
n = 50
outputs = [None]*n
pointers = [None]*n
memories = [[]]*n
input_queues = defaultdict(list)
outputs, pointers, memories = boot(outputs, pointers, memories, n)

# Run one after the other
while True:
    for i in range(len(memories)):
        destination, pointers[i], memories[i] = icc(memories[i], input_queues[i], pointers[i])
        if i == 0:
            print("Address", i, "Input:", input_queues[i], "Output:", destination)
        if destination:
            x, pointers[i], memories[i] = icc(memories[i], input_queues[i], pointers[i])
            y, pointers[i], memories[i] = icc(memories[i], input_queues[i], pointers[i])
        elif input_queues[i]:
            #print("Inputting at", i, "y", input_queues[i][0])
            destination, pointers[i], memories[i] = icc(memories[i], input_queues[i], pointers[i])
            destination = None
            if destination:
                x, pointers[i], memories[i] = icc(memories[i], input_queues[i], pointers[i])
                y, pointers[i], memories[i] = icc(memories[i], input_queues[i], pointers[i])
        if destination == 255:
            print(y)
            sys.exit()
        if destination:
            #print("Output at address", i, ": Destination:", destination, "x", x, "y", y)
            input_queues[destination].append(x)
            input_queues[destination].append(y)


Address 0 Input: [] Output: None
Address 0 Input: [] Output: 7
Address 0 Input: [] Output: 20
Address 0 Input: [] Output: 20
Address 0 Input: [] Output: 5
Address 0 Input: [] Output: 5
Address 0 Input: [] Output: 5
Address 0 Input: [] Output: None
Address 0 Input: [] Output: None
Address 0 Input: [] Output: None
Address 0 Input: [] Output: None
Address 0 Input: [] Output: None
Address 0 Input: [] Output: None
Address 0 Input: [] Output: None
Address 0 Input: [] Output: None
Address 0 Input: [] Output: None
Address 0 Input: [] Output: None
Address 0 Input: [] Output: None
19937


SystemExit: 

In [105]:
# Part 2
def boot(outputs, pointers, memories, n = 50):
    for i in range(n):
        #print("booting", i)
        outputs[i], pointers[i], memories[i] = icc(program, [i])#input_queues[i], i)
        #threads.append(threading.Thread(target=task).start())
    return outputs, pointers, memories

# Init network
n = 50
outputs = [None]*n
pointers = [None]*n
memories = [[]]*n
input_queues = defaultdict(list)
natvalues = []
outputs, pointers, memories = boot(outputs, pointers, memories, n)

# Run one after the other
sent = set()
while True:
    idle = True
    if any([input_queues[i] for i in input_queues]):
        idle = False
    for i in range(len(memories)):
        sav = input_queues[i].copy()
        destination, pointers[i], memories[i] = icc(memories[i], input_queues[i], pointers[i])
        if destination:
            x, pointers[i], memories[i] = icc(memories[i], input_queues[i], pointers[i])
            y, pointers[i], memories[i] = icc(memories[i], input_queues[i], pointers[i])
        elif input_queues[i]:
            #print("Inputting at", i, "y", input_queues[i][0])
            destination, pointers[i], memories[i] = icc(memories[i], input_queues[i], pointers[i])
            #destination = None
            if destination:
                x, pointers[i], memories[i] = icc(memories[i], input_queues[i], pointers[i])
                y, pointers[i], memories[i] = icc(memories[i], input_queues[i], pointers[i])
        if destination == 255:
            natvalues = [x, y]
            destination = None
        if destination:
            #print("Output at address", i, ": Destination:", destination, "x", x, "y", y)
            input_queues[destination].append(x)
            input_queues[destination].append(y)
            idle = False
    
    if idle and natvalues:
        input_queues[0].append(natvalues[0])
        input_queues[0].append(natvalues[1])
        print("NAT sent", natvalues)
        if y in sent:
            print("Part 2:", y)
            sys.exit()
            pass
        sent.add(y)

NAT sent [32887, 19937]
NAT sent [32887, 18319]
NAT sent [32887, 17045]
NAT sent [32887, 16094]
NAT sent [32887, 15405]
NAT sent [32887, 14915]
NAT sent [32887, 14569]
NAT sent [32887, 14326]
NAT sent [32887, 14155]
NAT sent [32887, 14035]
NAT sent [32887, 13951]
NAT sent [32887, 13893]
NAT sent [32887, 13852]
NAT sent [32887, 13823]
NAT sent [32887, 13803]
NAT sent [32887, 13789]
NAT sent [32887, 13779]
NAT sent [32887, 13772]
NAT sent [32887, 13767]
NAT sent [32887, 13764]
NAT sent [32887, 13762]
NAT sent [32887, 13760]
NAT sent [32887, 13759]
NAT sent [32887, 13758]
NAT sent [32887, 13758]
Part 2: 13758


SystemExit: 

In [99]:
o, p, m = icc(memories[0], [32887], pointers[0])
print(o)
o, p, m = icc(m, [19937], p)
print(o)

None
None


In [101]:
o, p, m = icc(m, [19937], p)
o

130742

In [78]:
pointers[0]

75

In [77]:
memories[0][:10]

[210, 21834, 1001, 62, 11, 10, 109, 2249, 105, 1]